## What-If Tool and SHAP on COMPAS keras model

This notebook shows:
- Training of a keras model on the [COMPAS](https://www.kaggle.com/danofer/compass) dataset.
- Explanation of inference results using [SHAP](https://github.com/slundberg/shap).
- Use of What-If Tool on the trained model, including SHAP values.

For ML fairness background on COMPAS see:

- https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing
- https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm
- http://www.crj.org/assets/2017/07/9_Machine_bias_rejoinder.pdf

This notebook trains a model to mimic the behavior of the COMPAS recidivism classifier and uses the SHAP library to provide feature importance for each prediction by the model. We can then analyze our COMPAS proxy model for fairness using the What-If Tool, and explore how important each feature was to each prediction through the SHAP values.

The specific binary classification task for this model is to determine if a person belongs in the "Low" risk class according to COMPAS (negative class), or the "Medium" or "High" risk class (positive class). We then analyze it with the What-If Tool for its ability to predict recidivism within two years of arrest.

A simpler version of this notebook that doesn't make use of the SHAP explainer can be found [here](https://colab.research.google.com/github/pair-code/what-if-tool/blob/master/WIT_COMPAS.ipynb).

Copyright 2019 Google LLC.
SPDX-License-Identifier: Apache-2.0

In [1]:
%autoreload 2
%load_ext autoreload
%matplotlib widget

import os, pickle, pdb, shutil, re, math
from copy import deepcopy, copy
from pathlib import Path

from tqdm.notebook import tqdm
import pandas as pd, numpy as np, torch
from sklearn.utils import shuffle
import torchvision.transforms as T
from torch.utils.tensorboard import SummaryWriter

from utils import lime_fit, sample_around

DTYPE, DEVICE = torch.float32, torch.device("cuda")
TOPTS = dict(dtype=DTYPE, device=DEVICE)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv(Path("") / "data" / "compas" / "cox-violent-parsed_filt.csv")

# Preprocess the data

# Filter out entries with no indication of recidivism or no compass score
df = df[df["is_recid"] != -1]
df = df[df["decile_score"] != -1]

# Rename recidivism column
df["recidivism_within_2_years"] = df["is_recid"]

# Make the COMPASS label column numeric (0 and 1), for use in our model
df["COMPASS_determination"] = np.where(df["score_text"] == "Low", 0, 1)

df = pd.get_dummies(df, columns=["sex", "race"])

# Get list of all columns from the dataset we will use for model input or output.
input_features = [
    "sex_Female",
    "sex_Male",
    "age",
    "race_African-American",
    "race_Caucasian",
    "race_Hispanic",
    "race_Native American",
    "race_Other",
    "priors_count",
    "juv_fel_count",
    "juv_misd_count",
    "juv_other_count",
]

to_keep = input_features + ["recidivism_within_2_years", "COMPASS_determination"]

to_remove = [col for col in df.columns if col not in to_keep]
df = df.drop(columns=to_remove)

input_columns = df.columns.tolist()
labels = df["COMPASS_determination"]
df.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,recidivism_within_2_years,COMPASS_determination,sex_Female,sex_Male,race_African-American,race_Caucasian,race_Hispanic,race_Native American,race_Other
0,69,0,0,0,0,0,0,0,1,0,0,0,0,1
1,69,0,0,0,0,0,0,0,1,0,0,0,0,1
3,34,0,0,0,0,1,0,0,1,1,0,0,0,0
4,24,0,0,1,4,1,0,0,1,1,0,0,0,0
5,24,0,0,1,4,1,0,0,1,1,0,0,0,0


In [3]:
# Create data structures needing for training and testing.
# The training data doesn't contain the column we are predicting,
# 'COMPASS_determination', or the column we are using for evaluation of our
# trained model, 'recidivism_within_2_years'.
df_for_training = df.drop(columns=["COMPASS_determination", "recidivism_within_2_years"])
train_size = int(len(df_for_training) * 0.8)

train_data = df_for_training[:train_size]
train_labels = labels[:train_size]
test_data = df_for_training[train_size:]
test_labels = labels[train_size:]

test_data_with_labels = df[train_size:]

In [4]:
Xtr, Ytr = train_data.values, train_labels.values
Xts, Yts = test_data.values, test_labels.values
MU, STD = np.mean(Xtr, -2), np.std(Xtr, -2)
normalize_fn = lambda x: (x - MU[None, ...]) / STD[None, ...]
Xtr = normalize_fn(Xtr)
Xts = normalize_fn(Xts)
train_loader = torch.utils.data.DataLoader(list(zip(*[Xtr, Ytr])), batch_size=1024, num_workers=8)
test_loader = torch.utils.data.DataLoader(list(zip(*[Xts, Yts])), batch_size=1024, num_workers=8)

In [7]:
# Create the model

# This is the size of the array we'll be feeding into our model for each example
input_size = len(train_data.iloc[0])

RACE_IDX = [i for (i, z) in enumerate(train_data.columns) if re.match(r"race_.*", z) is not None]

# activation = torch.nn.ReLU()
activation = torch.nn.Softplus(1e2)


def generate_model():
    model = (
        torch.nn.Sequential(
            torch.nn.Linear(input_size, 128),
            copy(activation),
            torch.nn.Linear(128, 128),
            copy(activation),
            torch.nn.Linear(128, 1),
            torch.nn.Sigmoid(),
        )
        .to(DTYPE)
        .to(DEVICE)
    )
    loss_obj = torch.nn.BCELoss()
    lam = 1e-3
    loss_fn = lambda Yp, Y: loss_obj(Yp, Y) + sum(lam * torch.sum(param**2) / 2 for param in model.parameters())

    x0 = torch.as_tensor(Xtr[torch.randint(0, Xtr.shape[0], size=(1000,)), :], device=DEVICE, dtype=DTYPE)
    Xs = sample_around(x0, torch.tensor(STD, device=DEVICE, dtype=DTYPE), N=int(1e2), alf=1e-2)
    Xs = Xs.transpose(0, 1)
    loss_fn_ = loss_fn

    def loss_fn(Yp, Y, penalize=True, penalty="exact", gam=1e1):
        ret = loss_fn_(Yp, Y)
        if penalize:
            Yp = model(Xs)
            W, b = lime_fit(Xs, Yp)
            if penalty == "exact":
                ret = ret + gam * torch.mean(torch.norm(W[..., RACE_IDX, 0], dim=-1))
            elif penalty == "mse":
                ret = ret + gam * torch.mean(torch.norm(W[..., RACE_IDX, 0], dim=-1) ** 2)
            elif penalty == "super-exact":
                ret = ret + gam * torch.mean(torch.sqrt(torch.norm(W[..., RACE_IDX, 0], dim=-1)))
            else:
                raise ValueError(f"penalty [{penalty}] is not supported")
        return ret

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.3)
    return model, loss_fn, optimizer, scheduler, Xs


def accuracy(model, loader):
    correct = 0
    for X, Y in loader:
        X, Y = X.to(DTYPE).to(DEVICE), Y.to(DEVICE)
        Yp = model(X)
        correct += torch.sum((Yp > 0.5).reshape(Y.shape) == Y).detach()
    return correct / len(loader.dataset)

In [15]:
accs, metrics = [], []
for (penalize, penalty, gam) in [
    (False, "exact", 1e1),
    (True, "exact", 1e1),
    (True, "mse", 1e2),
    (True, "super-exact", 1.5e0),
]:
    model, loss_fn, optimizer, scheduler, Xs = generate_model()
    shutil.rmtree(Path("") / "runs")
    writer = SummaryWriter()
    rng = tqdm(range(int(10)))
    for epoch in rng:
        for (i, (X, Y)) in enumerate(train_loader):
            X, Y = X.to(DTYPE).to(DEVICE), Y.to(DTYPE).to(DEVICE)
            loss = loss_fn(model(X).reshape(Y.shape), Y, penalize=penalize, penalty=penalty, gam=gam)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # rng.set_description(f"{loss.detach():.4e}")
            writer.add_scalar("loss/train", float(loss), i + epoch * len(train_loader))
            writer.add_scalar("step_size", float(optimizer.param_groups[0]["lr"]), i + epoch * len(train_loader))
            writer.flush()
        scheduler.step()
        # tqdm.write(f"Accuracy = {1e2 * accuracy(model, test_loader):.3f}%")
        rng.set_description(
            f"Accuracy = (test = {1e2 * accuracy(model, test_loader):.3f}%,"
            + f"train = {1e2 * accuracy(model, train_loader):.3f}%)"
        )
        # tqdm.write(f"Loss =     {loss_obj(model(X).reshape(Y.shape), Y):.5e}")

    W, b = lime_fit(Xs, model(Xs))
    print(f"Penalize = {penalize}")
    if penalize:
        print(f"Penalty = {penalty}")
    metric = torch.mean(torch.norm(W[..., RACE_IDX, 0], dim=-1))
    print(float(metric))
    accs.append(float(1e2 * accuracy(model, test_loader)))
    metrics.append(float(metric))
    print("#" * 80)

  0%|          | 0/10 [00:00<?, ?it/s]

Penalize = False
0.3325492739677429
################################################################################


  0%|          | 0/10 [00:00<?, ?it/s]

Penalize = True
Penalty = exact
0.00370749831199646
################################################################################


  0%|          | 0/10 [00:00<?, ?it/s]

Penalize = True
Penalty = mse
0.009267126210033894
################################################################################


  0%|          | 0/10 [00:00<?, ?it/s]

Penalize = True
Penalty = super-exact
0.0019427312072366476
################################################################################


In [18]:
print(accs)
print(metrics)

[74.11326599121094, 72.19679260253906, 73.14073181152344, 70.16590118408203]
[0.3325492739677429, 0.00370749831199646, 0.009267126210033894, 0.0019427312072366476]


In [19]:
def check_gam(gam):
    model, loss_fn, optimizer, scheduler, Xs = generate_model()
    shutil.rmtree(Path("") / "runs")
    writer = SummaryWriter()
    rng = tqdm(range(int(10)))
    for epoch in rng:
        for (i, (X, Y)) in enumerate(train_loader):
            X, Y = X.to(DTYPE).to(DEVICE), Y.to(DTYPE).to(DEVICE)
            loss = loss_fn(model(X).reshape(Y.shape), Y, penalize=True, penalty="exact", gam=gam)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            writer.add_scalar("loss/train", float(loss), i + epoch * len(train_loader))
            writer.add_scalar("step_size", float(optimizer.param_groups[0]["lr"]), i + epoch * len(train_loader))
            writer.flush()
        scheduler.step()
        # tqdm.write(f"Accuracy = {1e2 * accuracy(model, test_loader):.3f}%")
        rng.set_description(
            f"Accuracy = (test = {1e2 * accuracy(model, test_loader):.3f}%,"
            + f"train = {1e2 * accuracy(model, train_loader):.3f}%)"
        )
        # tqdm.write(f"Loss =     {loss_obj(model(X).reshape(Y.shape), Y):.5e}")

    W, b = lime_fit(Xs, model(Xs))
    print(f"Penalize = {penalize}")
    if penalize:
        print(f"Penalty = {penalty}")
        print(f"Gam = {gam}")
    metric = torch.mean(torch.norm(W[..., RACE_IDX, 0], dim=-1))
    print(metric)
    print("#" * 80)
    return metric

In [20]:
gams = 10.0 ** np.linspace(-2, 2, 10)
vals = [check_gam(gam) for gam in gams]

  0%|          | 0/10 [00:00<?, ?it/s]

Penalize = True
Penalty = super-exact
Gam = 0.01
tensor(0.2527, device='cuda:0', grad_fn=<MeanBackward0>)
################################################################################


  0%|          | 0/10 [00:00<?, ?it/s]

Penalize = True
Penalty = super-exact
Gam = 0.027825594022071243
tensor(0.1842, device='cuda:0', grad_fn=<MeanBackward0>)
################################################################################


  0%|          | 0/10 [00:00<?, ?it/s]

...

Penalize = True
Penalty = super-exact
Gam = 12.915496650148826
tensor(0.0025, device='cuda:0', grad_fn=<MeanBackward0>)
################################################################################


  0%|          | 0/10 [00:00<?, ?it/s]

Penalize = True
Penalty = super-exact
Gam = 35.93813663804626
tensor(0.0006, device='cuda:0', grad_fn=<MeanBackward0>)
################################################################################


  0%|          | 0/10 [00:00<?, ?it/s]

Penalize = True
Penalty = super-exact
Gam = 100.0
tensor(0.0004, device='cuda:0', grad_fn=<MeanBackward0>)
################################################################################


In [23]:
vals

[0.2527238726615906,
 0.18422257900238037,
 0.11337780207395554,
 0.06382934749126434,
 0.03317005932331085,
 0.014389969408512115,
 0.006740198004990816,
 0.002512869890779257,
 0.0006253707688301802,
 0.00040408188942819834]

In [25]:
gams

array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])

In [24]:
accs = [73.455, 73.713, 73.627, 73.827, 73.513, 73.169, 72.912, 72.311, 55.921, 54.634]

--------------------------------------------------------------------------------

In [34]:
# Create a SHAP explainer by passing a subset of our training data
import shap

explainer = shap.DeepExplainer(model, torch.tensor(train_data.values[:200]).to(DTYPE).to(DEVICE))
explainer.explainer.interim

False

In [31]:
# Explain predictions of the model on the first 5 examples from our training set
# to test the SHAP explainer.
shap_values = explainer.shap_values(torch.tensor(train_data.values[:5]).to(DTYPE).to(DEVICE))
shap_values
# print(shap_values[0, :])

array([[-5.71903102e-02, -1.97675225e-04, -1.98997383e-04,
        -1.92532389e-04, -4.12244089e-02, -1.43817724e-09,
         1.28570010e-09, -1.71174586e-04,  1.77759330e-06,
         3.25610164e-11,  0.00000000e+00, -3.68765322e-04],
       [-5.71903102e-02, -1.97675225e-04, -1.98997383e-04,
        -1.92532389e-04, -4.12244089e-02, -1.43817724e-09,
         1.28570010e-09, -1.71174586e-04,  1.77759330e-06,
         3.25610164e-11,  0.00000000e+00, -3.68765322e-04],
       [-6.04670588e-03, -3.85728083e-04, -3.56173958e-04,
        -4.00341756e-04, -8.20497125e-02, -4.35711378e-09,
         2.81565038e-09,  2.28080557e-06,  3.80985762e-06,
         1.35330511e-10,  0.00000000e+00,  1.81077341e-11],
       [ 5.16895279e-02, -7.97657587e-04, -5.81035041e-04,
        -3.81269056e-04, -1.27719998e-01, -1.18468336e-07,
        -3.39043034e-08,  8.59804459e-06,  1.11365889e-05,
         9.25007129e-08,  0.00000000e+00,  1.37961976e-07],
       [ 5.16895279e-02, -7.97657587e-04, -5.8103504

--------------------------------------------------------------------------------

In [ ]:
# @title Show model results and SHAP values in WIT
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

num_datapoints = 1000  # @param {type: "number"}

# Column indices to strip out from data from WIT before passing it to the model.
columns_not_for_model_input = [
    test_data_with_labels.columns.get_loc("recidivism_within_2_years"),
    test_data_with_labels.columns.get_loc("COMPASS_determination"),
]

# Return model predictions and SHAP values for each inference.
def custom_predict_with_shap(examples_to_infer):
    # Delete columns not used by model
    model_inputs = np.delete(np.array(examples_to_infer), columns_not_for_model_input, axis=1)

    # Get the class predictions from the model.
    preds = model.predict(model_inputs)
    preds = [[1 - pred[0], pred[0]] for pred in preds]

    # Get the SHAP values from the explainer and create a map of feature name
    # to SHAP value for each example passed to the model.
    shap_output = explainer.shap_values(np.array(model_inputs))[0]
    attributions = []
    for shap in shap_output:
        attrs = {}
        for i, col in enumerate(df_for_training.columns):
            attrs[col] = shap[i]
        attributions.append(attrs)
    ret = {"predictions": preds, "attributions": attributions}
    return ret


examples_for_shap_wit = test_data_with_labels.values.tolist()
column_names = test_data_with_labels.columns.tolist()

config_builder = (
    WitConfigBuilder(examples_for_shap_wit[:num_datapoints], feature_names=column_names)
    .set_custom_predict_fn(custom_predict_with_shap)
    .set_target_feature("recidivism_within_2_years")
)

ww = WitWidget(config_builder, height=800)

#### What-If Tool exploration ideas

- Organize datapoints by "inference score" (can do this through binning or use of scatter plot) to see points ordered by how likely they were determined to re-offend.
  - Select a point near the boundary line (where red points turn to blue points)
  - Find the nearest counterfactual to see a similar person with a different decision. What is different?
  - Look at the partial dependence plots for the selected person. What changes in what features would change the decision on this person?
- Explore the attribution values provided by SHAP.
  - For a variety of selected datapoints, look at which features have the highest positive attribution values. These are making the model predict higher risk for this person.
  - Look at which features have the lowest negative attribution values as well. These are making the model predict lower risk for this person.
  - How well do these attribution scores line up with the partial dependence plots for those datapoints?
  - Use the attribution scores in the datapoints visualizations to look for interesting patterns. As one example, you could set the scatter X-axis to "attributions__age" and the scatter Y-axis to "attributions__priors_count" with the points colored by "Inference score" to investigate the relationship between feature importance of those two features and how those relate to the score the model gives for each datapoint being "High risk".
- In "Performance and Fairness" tab, slice the dataset by different features (such as race or sex)
  - Look at the confusion matrices for each slice - How does performance compare in those slices? What from the training data may have caused the difference in performance between the slices? What root causes could exist?
  - Use the threshold optimization buttons to optimize positive classification thresholds for each slice based on any of the possible fairness constraints - How different do the thresholds have to be to achieve that constraint? How varied are the thresholds depending on the fairness constraint chosen?

- In the "Performance + Fairness" tab, change the cost ratio so that you can optimize the threshold based off of a non-symmetric cost of false positives vs false negatives. Then click the "optimize threshold" button and see the effect on the confusion matrix. 
  - Slice the dataset by a feature, such as sex or race. How has the new cost ratio affected the disparity in performance between slices? Click the different threshold optimization buttons to see how the changed cost ratio affects the disparity given different fairness constraints.



#### Further exploration ideas

- Edit the training data so that race fields are not included as a feature and train a new model with this data as input (make sure to create a new explainer and a new custom prediction function that filters race out of model input and uses the right explainer and model).
- Load the new model with set_compare_custom_predict_fn and compare it with the original model.
  - HINT: You'll need to make edits in 3 separate code cells.
  - Is there still a racial disparity in model results? If so, what could be the causes?
  - How did the SHAP attributions change?

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f99e494-88c0-4d9d-8943-c19d4dba913c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>